### Import Libraries

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
from scipy.stats import zscore
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from matplotlib.pyplot import figure

In [2]:
msk_impact_clinical_data = pd.read_csv('./Data/CBioportal/msk_impact_2017_clinical_data.tsv',sep='\t')

### Inspect Datasets

In [3]:
msk_impact_clinical_data.head(3)

,Study ID,Patient ID,Sample ID,Cancer Type,Cancer Type Detailed,DNA Input,Fraction Genome Altered,Matched Status,Metastatic Site,Mutation Count,...,Sample coverage,Sample Type,Sex,Smoking History,Somatic Status,Specimen Preservation Type,Specimen Type,TMB (nonsynonymous),Tumor Purity,Patient's Vital Status
0,msk_impact_2017,P-0000004,P-0000004-T01-IM3,Breast Cancer,Breast Invasive Ductal Carcinoma,250.0,0.2782,Matched,NaN,16,...,428,Primary,Female,Unknown,Matched,FFPE,Biopsy,17.746485,50.0,ALIVE
1,msk_impact_2017,P-0000015,P-0000015-T01-IM3,Breast Cancer,Breast Invasive Ductal Carcinoma,198.0,0.3503,Matched,Liver,7,...,281,Metastasis,Female,Never,Matched,FFPE,Biopsy,7.764087,40.0,DECEASED
2,msk_impact_2017,P-0000023,P-0000023-T01-IM3,Mesothelioma,Peritoneal Mesothelioma,250.0,0.1596,Matched,NaN,5,...,454,Primary,Male,Never,Matched,FFPE,Biopsy,5.545777,30.0,DECEASED


In [4]:
msk_impact_clinical_data.columns

Index(['Study ID', 'Patient ID', 'Sample ID', 'Cancer Type',
       'Cancer Type Detailed', 'DNA Input', 'Fraction Genome Altered',
       'Matched Status', 'Metastatic Site', 'Mutation Count', 'Oncotree Code',
       'Overall Survival (Months)', 'Overall Survival Status',
       'Primary Tumor Site', 'Sample Class', 'Sample Collection Source',
       'Number of Samples Per Patient', 'Sample coverage', 'Sample Type',
       'Sex', 'Smoking History', 'Somatic Status',
       'Specimen Preservation Type', 'Specimen Type', 'TMB (nonsynonymous)',
       'Tumor Purity', 'Patient's Vital Status'],
      dtype='object')

In [5]:
data_clinical_patient = pd.read_csv('./Data/CBioportal/msk_impact_2017/data_clinical_patient.txt',\
                             sep='\t',\
                             skiprows = 4,\
                             low_memory=False)

In [6]:
data_clinical_patient.head(2)

,PATIENT_ID,SEX,VITAL_STATUS,SMOKING_HISTORY,OS_MONTHS,OS_STATUS
0,P-0000004,Female,ALIVE,Unknown,NaN,0:LIVING
1,P-0000015,Female,DECEASED,Never,NaN,1:DECEASED


In [7]:
data_clinical_patient.columns

Index(['PATIENT_ID', 'SEX', 'VITAL_STATUS', 'SMOKING_HISTORY', 'OS_MONTHS',
       'OS_STATUS'],
      dtype='object')

In [8]:
data_clinical_sample = pd.read_csv('./Data/CBioportal/msk_impact_2017/data_clinical_sample.txt',\
                             sep='\t',\
                             skiprows = 4,\
                             low_memory=False)

In [9]:
data_clinical_sample.head(3)

,PATIENT_ID,SAMPLE_ID,SAMPLE_COLLECTION_SOURCE,SPECIMEN_PRESERVATION_TYPE,SPECIMEN_TYPE,DNA_INPUT,SAMPLE_COVERAGE,TUMOR_PURITY,MATCHED_STATUS,SAMPLE_TYPE,PRIMARY_SITE,METASTATIC_SITE,SAMPLE_CLASS,ONCOTREE_CODE,CANCER_TYPE,CANCER_TYPE_DETAILED,SOMATIC_STATUS,TMB_NONSYNONYMOUS
0,P-0000004,P-0000004-T01-IM3,Outside,FFPE,Biopsy,250.0,428,50.0,Matched,Primary,Breast,Not Applicable,Tumor,IDC,Breast Cancer,Breast Invasive Ductal Carcinoma,Matched,17.746485
1,P-0000015,P-0000015-T01-IM3,In-House,FFPE,Biopsy,198.0,281,40.0,Matched,Metastasis,Breast,Liver,Tumor,IDC,Breast Cancer,Breast Invasive Ductal Carcinoma,Matched,7.764087
2,P-0000023,P-0000023-T01-IM3,In-House,FFPE,Biopsy,250.0,454,30.0,Matched,Primary,Peritoneum,Not Applicable,Tumor,PEMESO,Mesothelioma,Peritoneal Mesothelioma,Matched,5.545777


In [10]:
data_clinical_sample.columns

Index(['PATIENT_ID', 'SAMPLE_ID', 'SAMPLE_COLLECTION_SOURCE',
       'SPECIMEN_PRESERVATION_TYPE', 'SPECIMEN_TYPE', 'DNA_INPUT',
       'SAMPLE_COVERAGE', 'TUMOR_PURITY', 'MATCHED_STATUS', 'SAMPLE_TYPE',
       'PRIMARY_SITE', 'METASTATIC_SITE', 'SAMPLE_CLASS', 'ONCOTREE_CODE',
       'CANCER_TYPE', 'CANCER_TYPE_DETAILED', 'SOMATIC_STATUS',
       'TMB_NONSYNONYMOUS'],
      dtype='object')

In [11]:
data_cna = pd.read_csv('./Data/CBioportal/msk_impact_2017/data_cna.txt',\
                             sep='\t',\
                             low_memory=False)

In [12]:
data_cna.head(3)

,Hugo_Symbol,P-0000004-T01-IM3,P-0000015-T01-IM3,P-0000023-T01-IM3,P-0000024-T01-IM3,P-0000025-T02-IM5,P-0000025-T01-IM3,P-0000026-T01-IM3,P-0000027-T01-IM3,P-0000030-T01-IM3,...,P-0012404-T01-IM5,P-0012405-T01-IM5,P-0012406-T01-IM5,P-0012423-T01-IM5,P-0012434-T01-IM5,P-0012442-T01-IM5,P-0012500-T01-IM5,P-0012524-T01-IM5,P-0012557-T01-IM5,P-0012622-T01-IM5
0,ABL1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ACVR1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AKT1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
data_cna.columns

Index(['Hugo_Symbol', 'P-0000004-T01-IM3', 'P-0000015-T01-IM3',
       'P-0000023-T01-IM3', 'P-0000024-T01-IM3', 'P-0000025-T02-IM5',
       'P-0000025-T01-IM3', 'P-0000026-T01-IM3', 'P-0000027-T01-IM3',
       'P-0000030-T01-IM3',
       ...
       'P-0012404-T01-IM5', 'P-0012405-T01-IM5', 'P-0012406-T01-IM5',
       'P-0012423-T01-IM5', 'P-0012434-T01-IM5', 'P-0012442-T01-IM5',
       'P-0012500-T01-IM5', 'P-0012524-T01-IM5', 'P-0012557-T01-IM5',
       'P-0012622-T01-IM5'],
      dtype='object', length=10946)

In [14]:
data_cna_hg19 = pd.read_csv('./Data/CBioportal/msk_impact_2017/data_cna_hg19.seg',\
                             sep='\t',\
                             low_memory=False)

In [15]:
data_cna_hg19.head(3)

,ID,chrom,loc.start,loc.end,num.mark,seg.mean
0,P-0001658-T01-IM3,1,2488138,117753462,302,-0.1531
1,P-0001658-T01-IM3,1,118145461,243708856,235,0.1233
2,P-0001658-T01-IM3,1,243716138,245977996,11,-0.2639


In [16]:
data_cna_hg19.columns

Index(['ID', 'chrom', 'loc.start', 'loc.end', 'num.mark', 'seg.mean'], dtype='object')

In [17]:
data_gene_panel_matrix = pd.read_csv('./Data/CBioportal/msk_impact_2017/data_gene_panel_matrix.txt',\
                             sep='\t',\
                             low_memory=False)

In [18]:
data_gene_panel_matrix.head(3)

,SAMPLE_ID,mutations,cna,structural_variants
0,P-0000004-T01-IM3,IMPACT341,IMPACT341,IMPACT341
1,P-0000015-T01-IM3,IMPACT341,IMPACT341,IMPACT341
2,P-0000023-T01-IM3,IMPACT341,IMPACT341,IMPACT341


In [19]:
data_gene_panel_matrix.columns

Index(['SAMPLE_ID', 'mutations', 'cna', 'structural_variants'], dtype='object')

In [20]:
data_mutations = pd.read_csv('./Data/CBioportal/msk_impact_2017/data_mutations.txt',\
                             sep='\t',\
                             skiprows=1,\
                             low_memory=False)

In [21]:
data_mutations.head(3)

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,...,n_alt_count,HGVSc,HGVSp,HGVSp_Short,Transcript_ID,RefSeq,Protein_position,Codons,Hotspot,cDNA_change
0,SPEN,NaN,NaN,GRCh37,1,16265908,16265908,+,missense_variant,Missense_Mutation,...,NaN,ENST00000375759.3:c.10981A>T,p.Ile3661Phe,p.I3661F,ENST00000375759,NM_015001.2,3661.0,Att/Ttt,0,c.10981A>T
1,ALK,NaN,NaN,GRCh37,2,29543736,29543736,+,missense_variant,Missense_Mutation,...,NaN,ENST00000389048.3:c.1427T>C,p.Val476Ala,p.V476A,ENST00000389048,NM_004304.4,476.0,gTg/gCg,0,c.1427T>C
2,PDCD1,NaN,NaN,GRCh37,2,242793433,242793433,+,missense_variant,Missense_Mutation,...,NaN,ENST00000334409.5:c.644C>T,p.Ala215Val,p.A215V,ENST00000334409,NM_005018.2,215.0,gCc/gTc,0,c.644C>T


In [22]:
data_sv = pd.read_csv('./Data/CBioportal/msk_impact_2017/data_sv.txt',\
                             sep='\t',\
                             low_memory=False)

In [23]:
data_sv.head(3)

,Sample_Id,SV_Status,Site1_Chromosome,Site1_Description,Site1_Region,Site1_Region_Number,Site1_Hugo_Symbol,Site1_Ensembl_Transcript_Id,Site1_Position,Site2_Chromosome,...,Tumor_Variant_Count,Annotation,Breakpoint_Type,Comments,Normal_Read_Count,Normal_Variant_Count,Normal_Paired_End_Read_Count,Normal_Split_Read_Count,SV_Length,SV_VariantId
0,P-0000047-T01-IM3,SOMATIC,6,Exon 26 of ROS1(-),Exon,NaN,ROS1,NaN,117674261.0,6,...,52.0,NaN,35332045-N_bc40,NaN,185633.0,0.0,19.0,68.0,41381597.0,94059.0
1,P-0000058-T01-IM3,SOMATIC,2,Intron of ALK(-): 452bp before exon 20,Intron,NaN,ALK,NaN,29446846.0,2,...,3.0,NaN,35399162-N_bc20,NaN,461680.0,0.0,6.0,47.0,84914142.0,94065.0
2,P-0000187-T01-IM3,SOMATIC,13,Exon 3 of RB1(+),Exon,NaN,RB1,NaN,48916784.0,1,...,47.0,NaN,PRECISE,NaN,0.0,0.0,52.0,62.0,0.0,94338.0


In [24]:
data_sv.columns

Index(['Sample_Id', 'SV_Status', 'Site1_Chromosome', 'Site1_Description',
       'Site1_Region', 'Site1_Region_Number', 'Site1_Hugo_Symbol',
       'Site1_Ensembl_Transcript_Id', 'Site1_Position', 'Site2_Chromosome',
       'Site2_Description', 'Site2_Region', 'Site2_Region_Number',
       'Site2_Hugo_Symbol', 'Site2_Ensembl_Transcript_Id', 'Site2_Position',
       'Site2_Effect_on_Frame', 'Class', 'NCBI_Build', 'Connection_Type',
       'Event_Info', 'DNA_Support', 'Tumor_Read_Count', 'Tumor_Variant_Count',
       'Annotation', 'Breakpoint_Type', 'Comments', 'Normal_Read_Count',
       'Normal_Variant_Count', 'Normal_Paired_End_Read_Count',
       'Normal_Split_Read_Count', 'SV_Length', 'SV_VariantId'],
      dtype='object')

In [25]:
supplimentary_info = pd.read_csv('./Data/Supplimentary_info.csv',\
                             skiprows = 0,\
                             low_memory=False)

In [26]:
supplimentary_info = supplimentary_info.rename(columns={'Assay_ID': 'SAMPLE_ID'})

In [27]:
supplimentary_info.head(2)

,Patient_ID,Sample_ID,SAMPLE_ID,Status,SampleType,AssayPerformance,SpecimenType,DNAInput_ng,Coverage,TumorPurity,...,Gender,ExonicMutationCount,SNVCount,INDELCount,SCNACount,SVCount,SilentMutationCount,VitalStatus,SmokingStatus,TissueAge_yrs
0,P-0000004,P-0000004-T01,P-0000004-T01-IM3,Outside,FFPE,Success,Biopsy,250.0,428,50.0,...,Female,16,15,1,3,0,17,ALIVE,Unknown,na
1,P-0000015,P-0000015-T01,P-0000015-T01-IM3,In-House,FFPE,Success,Biopsy,198.0,281,40.0,...,Female,7,6,1,11,0,7,DECEASED,Never,na


In [28]:
tumor_types = pd.read_csv('./Data/tumor_types.txt',\
                             sep='\t',\
                             low_memory=False)

In [29]:
tumor_types.head(2)

,CANCER_TYPE,CANCER_TYPE_DETAILED,N,Cancer_Type
0,Adrenocortical Carcinoma,Adrenocortical Adenoma,1,NaN
1,Adrenocortical Carcinoma,Adrenocortical Carcinoma,25,NaN


### Merging Relevant Tables

In [30]:
data_clinical = pd.merge(data_clinical_sample,\
                         data_clinical_patient,\
                         on='PATIENT_ID')

In [31]:
data_clinical = pd.merge(data_clinical,\
                         supplimentary_info,\
                         on='SAMPLE_ID')

In [32]:
data_table = pd.merge(data_clinical,\
                      tumor_types,\
                      left_on=["CANCER_TYPE"],\
                      right_on = ["CANCER_TYPE_DETAILED"])

In [33]:
data_table = pd.merge(data_table,\
                data_mutations,\
                left_on = ["SAMPLE_ID"],\
                right_on = ["Tumor_Sample_Barcode"])

In [34]:
data_table.head(2)

,PATIENT_ID,SAMPLE_ID,SAMPLE_COLLECTION_SOURCE,SPECIMEN_PRESERVATION_TYPE,SPECIMEN_TYPE,DNA_INPUT,SAMPLE_COVERAGE,TUMOR_PURITY,MATCHED_STATUS,SAMPLE_TYPE,...,n_alt_count,HGVSc,HGVSp,HGVSp_Short,Transcript_ID,RefSeq,Protein_position,Codons,Hotspot,cDNA_change
0,P-0000030,P-0000030-T01-IM3,Outside,FFPE,Biopsy,250.0,757,40.0,Unmatched,Metastasis,...,NaN,ENST00000371953.3:c.517C>T,p.Arg173Cys,p.R173C,ENST00000371953,NM_000314.4,173.0,Cgc/Tgc,0,c.517C>T
1,P-0000030,P-0000030-T01-IM3,Outside,FFPE,Biopsy,250.0,757,40.0,Unmatched,Metastasis,...,NaN,ENST00000269305.4:c.626_627delGA,p.Arg209LysfsTer6,p.R209Kfs*6,ENST00000269305,NM_001126112.2,209.0,aGA/a,0,c.626_627delGA


### Drop duplicate columns after merging

In [46]:
columns_to_drop = ['AssayPerformance',\
                   'Cancer_Type',\
                   'Coverage',\
                   'DNA_INPUT',\
                   'Gender',\
                   'MATCHED_STATUS',\
                   'MatchStatus',\
                   'METASTATIC_SITE',\
                   'OS_MONTHS',\
                   'OS_STATUS',\
                   'Patient_ID',\
                   'PATIENT_ID',\
                   'PRIMARY_SITE',\
                   'SAMPLE_CLASS',\
                   'SAMPLE_COLLECTION_SOURCE',\
                   'Sample_ID',\
                   'SampleType.1',\
                   'SmokingStatus',\
                   'SPECIMEN_PRESERVATION_TYPE',\
                   'SpecimenType',\
                   'Status',\
                   'TumorPurity',\
                   'VITAL_STATUS',\
                   'VitalStatus']

In [47]:
data_table = data_table.drop(columns_to_drop,axis=1)

In [ ]:
data_table.head(2)

In [ ]:
data_table.columns

### Export the Dataset of Modeling

In [39]:
data_table.to_csv('./processed_data/processed_data.csv', index=False)

### Exploratory Data Analysis Plots
target variable is 'SAMPLE_TYPE'

### Check for missing values

In [ ]:
fig, ax = plt.subplots( figsize = (15, 8))
sns.heatmap(data_table.isnull())
ax.set_title('Raw Dataframe')
plt.show()

In [ ]:
# Reference: Code from my self case study 1
total =  data_table.isnull().sum().sort_values(ascending = False)
percent = (data_table.isnull().sum() /  data_table.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis = 1, keys = ['Count_NaN', 'Percentage_Nan'])
missing_data.head(14)

### Drop columns having more than 5% of missing values

Overall Survival Months feature and Cancer type feature has missing values of 25% and 15%. Hence these features can be removed

In [ ]:
data_table = data_table.drop(['OS_MONTHS',\
                 'Cancer_Type'],axis=1)

In [ ]:
# Drop target variable if needed for correlation analysis
master_data = data_table.drop(['SAMPLE_TYPE'],axis=1)

### Feature Correlation Analysis

In [ ]:
# Find correaltion between features
corrMat = master_data.corr().values
plt.hist(corrMat.flatten(),20)
plt.xlabel('Pearson correlation coefficient')
plt.ylabel('Count')

In [ ]:
fig, axs = plt.subplots(figsize = (15, 10)) 
sns.heatmap(master_data.corr())
plt.title('Correlation between all Features')
plt.show()

It can be seen that certain features are highly correlated (positive/negative with each other)

### Baseline Performance

A baseline accuracy of 57.94% is expected

In [ ]:
data_table['SAMPLE_TYPE'].value_counts()

In [ ]:
data_table['SAMPLE_TYPE'].value_counts(normalize=True)

### Density Plots of Numerical Features w.r.t. Target Variable

In [ ]:
# Reference: Code from my self casestudy 1
def mydistplot1(variable, data):
    labelsize = 12
    plt.rc('font', family='serif')
    plt.rc('xtick', labelsize=labelsize)
    plt.rc('ytick', labelsize=labelsize)
    plt.rc('axes', labelsize=labelsize)
    fig, axs = plt.subplots(figsize=(4, 3), dpi=110)
    ax1 = axs.twinx()
    plt.subplots_adjust(hspace=.3)
    class_1 = data[data['SAMPLE_TYPE'] == "Primary"][variable]
    class_2 = data[data['SAMPLE_TYPE'] == "Metastasis"][variable]
    class_1.plot.kde(ax=ax1,c='b',label='Primary')
    class_2.plot.kde(ax=ax1,c='r',label='Metastasis')
#     class_1.plot.hist(ax=axs,bins=12, alpha=0.5,label='Primary')
#     class_2.plot.hist(ax=axs,bins=12, alpha=0.5,label='Metastasis')
    axs.set_xlabel(variable)
    axs.set_ylabel('Density')
    ax1.legend()

In [ ]:
numericalFeats = master_data.select_dtypes('number').columns
categoricalFeats = master_data.select_dtypes('object').columns

In [ ]:
for variable in tqdm(numericalFeats):
    mydistplot1(variable,data_table)

### Statistical Test on Numerical Features

In [ ]:
# Reference: Code from my self casestudy 1
statDF = pd.DataFrame()

for feature in numericalFeats:
    group1 = data_table[data_table['SAMPLE_TYPE'] == "Primary"]
    group2 = data_table[data_table['SAMPLE_TYPE'] == "Metastasis"]
    tstats, p_value = stats.ttest_ind(group1[feature], group2[feature])    
    statDF = pd.concat([statDF, pd.DataFrame.from_records([{'Feature': feature,\
                                                   't-statistics':tstats,\
                                                   'P_value': p_value}])])

In [ ]:
statDF

In [ ]:
statDF['Significant'] = statDF['P_value']<0.001
statDF.sort_values('P_value')

Three statistically significant features SCNACount, DNAInput_ng, SAMPLE_COVERAGE with an alpha level of 0.001 were foundout

In [ ]:
significantFeats = statDF[statDF['Significant']==True]['Feature'].tolist()

In [ ]:
significantFeats

### Violin Plots of Significant Features

In [ ]:
# Reference: Code from my self casestudy 1
def myviolinplot(feature, data):
    sns.violinplot(y=feature,\
                   x="SAMPLE_TYPE",\
                   data=data,\
                   palette="muted",\
                   split=True)
    plt.show()

In [ ]:
for variable in tqdm(significantFeats):
    myviolinplot(variable,data_table)

### TSNE Visualization

#### Convert Binry Target Variable to Numeric

In [ ]:
data_table['SAMPLE_TYPE'].replace(['Primary', 'Metastasis'],
                        [0, 1], inplace=True)

### Separate Numerical Data

In [ ]:
numericalFeats = data_table.select_dtypes('number').columns
categoricalFeats = data_table.select_dtypes('object').columns

numdata = data_table[numericalFeats]
numdata = numdata.fillna(numdata.mean())

In [ ]:
tsne = TSNE(n_components=2, perplexity=20.0, n_iter=2000,verbose=1)
z = tsne.fit_transform(numdata) 

In [ ]:
df = pd.DataFrame()
df["y"] = numdata['SAMPLE_TYPE']
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]
 
sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 2),
                data=df).set(title="MSK IMPACT data T-SNE projection")

### Outlier Detection

In [ ]:
# Compute Z-Scores
num_zscores = numdata.apply(zscore)
 
# Find all cells where zscore>3 or zscore<-3
num_zscores_binary = num_zscores.abs()>3
 
fig, axs = plt.subplots(figsize = (15, 10)) 
sns.heatmap(num_zscores_binary)
plt.title('Outlier data points across patients |z-score|>3')
plt.show()

### EDA of MSK Impact Clinical Data

In [ ]:
figure(figsize=(6, 10), dpi=200)
cancer_types_count = msk_impact_clinical_data['Cancer Type'].value_counts()
cancer_types_count.sort_values().plot(kind = 'barh')
plt.xlabel('Count')
plt.ylabel('Type of cancer')

In [ ]:
print('Number of Types of Cancers present in data: ', len(cancer_types_count.keys()))

In [ ]:
patient_ids = msk_impact_clinical_data['Patient ID'].tolist()
patient_id_unique = list(set(patient_ids))
print('Number of Patient IDs: ', len(patient_ids))
print('Number of Unique Patient IDs: ', len(patient_id_unique))
print('Number of redundand Patient IDs:', len(patient_ids)-len(patient_id_unique))

In [ ]:
sample_ids = msk_impact_clinical_data['Sample ID'].tolist()
sample_id_unique = list(set(sample_ids))
print('Number of Sample IDs: ', len(sample_ids))
print('Number of Unique Sample IDs: ', len(sample_id_unique))
print('Number of redundand Sample IDs:', len(sample_ids)-len(sample_id_unique))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
frac_genome_altered = msk_impact_clinical_data['Fraction Genome Altered']
frac_genome_altered.plot.hist(ax=ax,bins=12, alpha=0.5)
frac_genome_altered.plot.kde(ax=ax1,c='k')
ax.set_xlabel('Fraction of Genome altered')

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
matched_status = msk_impact_clinical_data['Matched Status'].value_counts()
matched_status.sort_values().plot(kind = 'barh')
ax.set_xlabel('Count')
ax.set_ylabel('Sample matching status')
for container in ax.containers:
    ax.bar_label(container)

In [ ]:
matched_percentage = matched_status*100/sum(matched_status)
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
matched_percentage.sort_values().plot(kind = 'barh')
ax.set_xlabel('Percentage')
ax.set_ylabel('Sample matching status')
for container in ax.containers:
    ax.bar_label(container)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
metastatic_site_count = msk_impact_clinical_data['Metastatic Site'].value_counts()
ax.set_title('Top 10 Metastatic sites')
metastatic_site_count[:10].sort_values().plot(kind = 'barh')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
mutation_count = msk_impact_clinical_data['Mutation Count']
plt.hist(mutation_count, bins=20, alpha=0.5)
ax.set_title('Mutation count histogram')
plt.xlabel('Number of mutations')
plt.ylabel('Count')
print('Mean of mutation count: ',np.mean(mutation_count))
print('Std. dev of mutation count: ',np.std(mutation_count))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
dna_input = msk_impact_clinical_data['DNA Input']
dna_input.plot.hist(ax=ax,bins=12, alpha=0.5)
dna_input.plot.kde(ax=ax1,c='k')
ax.set_xlabel('DNA Input')

#### Overall survival in months

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
survival_months = msk_impact_clinical_data['Overall Survival (Months)']
survival_months.plot.hist(ax=ax,bins=12, alpha=0.5)
survival_months.plot.kde(ax=ax1,c='k')
ax.set_xlabel('Overall Survival (Months)')

In [ ]:
print('Mean value of overall survival in months: ', np.mean(survival_months))
print('Std. dev of overall survival in months: ', np.std(survival_months))

#### Survival status of patients

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
survival_status = msk_impact_clinical_data['Overall Survival Status'].value_counts()
survival_status.sort_values().plot(kind = 'barh')
ax.set_title('Overall survival status')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)
    
survival_status_percentage = survival_status*100/sum(survival_status)
survival_status_percentage.sort_values().plot(kind = 'barh')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)

1. It can be seen that 71.55% of patients are alive and 28.45% patients are deceased

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
primary_tumor_site_count = msk_impact_clinical_data['Primary Tumor Site'].value_counts()
ax.set_title('Primary Tumor Site')
primary_tumor_site_count[:10].sort_values().plot(kind = 'barh')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
sample_collection_source = msk_impact_clinical_data['Sample Collection Source'].value_counts()
sample_collection_source.sort_values().plot(kind = 'barh')
ax.set_title('Sample Collection Source')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)
    
sample_collection_source_percentage = sample_collection_source*100/sum(sample_collection_source)
sample_collection_source_percentage.sort_values().plot(kind = 'barh')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)

It can be noted that 65.04% of samples are collected In-House and 34.96% of samples are collected outside home

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
samples_patient = msk_impact_clinical_data['Number of Samples Per Patient'].value_counts()
samples_patient.sort_values().plot(kind = 'barh')
ax.set_title('Number of samples taken from patients')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
samples_patient_percentage = samples_patient*100/sum(samples_patient)
samples_patient_percentage.sort_values().plot(kind = 'barh')
ax.set_title('Number of samples taken from patients')
ax.set_xlabel('Percentage')
for container in ax.containers:
    ax.bar_label(container)

1. Only one sample is collected from majority (89%) of patients

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
sample_coverage = msk_impact_clinical_data['Sample coverage']
sample_coverage.plot.hist(ax=ax,bins=12, alpha=0.5)
sample_coverage.plot.kde(ax=ax1,c='k')
ax.set_xlabel('Sample coverage')

In [ ]:
print('Mean value of sample coverage: ', np.mean(sample_coverage))
print('Std. dev of sample coverage: ', np.std(sample_coverage))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
sample_type = msk_impact_clinical_data['Sample Type'].value_counts()
sample_type.sort_values().plot(kind = 'barh')
ax.set_title('Sample type')
ax.set_xlabel('Count/Percentage')
for container in ax.containers:
    ax.bar_label(container)
    
sample_type_percentage = sample_type*100/sum(sample_type)
sample_type_percentage.sort_values().plot(kind = 'barh')
ax.set_xlabel('Count/Percentage')
for container in ax.containers:
    ax.bar_label(container)

1. It can be seen that 43.23% of samples are that of metastasis stage

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
sex = msk_impact_clinical_data['Sex'].value_counts()
sex.sort_values().plot(kind = 'barh')
ax.set_title('Gender of the patient')
ax.set_xlabel('Count/Percentage')
for container in ax.containers:
    ax.bar_label(container)
    
sex_percentage = sex*100/sum(sex)
sex_percentage.sort_values().plot(kind = 'barh')
ax.set_xlabel('Count/Percentage')
for container in ax.containers:
    ax.bar_label(container)

It can be seen that the gender distribution of patients is more or less equal (i.e. 50%-50%)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
smoking_history = msk_impact_clinical_data['Smoking History'].value_counts()
smoking_history.sort_values().plot(kind = 'barh')
ax.set_title('Smoking history of patients')
ax.set_xlabel('Count/percentage')
for container in ax.containers:
    ax.bar_label(container)
    
smoking_history_percentage = smoking_history*100/sum(smoking_history)
smoking_history_percentage.sort_values().plot(kind = 'barh')
ax.set_xlabel('Count/percentage')
for container in ax.containers:
    ax.bar_label(container)

1. The data shows 39.89% of patients are having the habbit of smoking or used to smoke before

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
somatic_status = msk_impact_clinical_data['Somatic Status'].value_counts()
somatic_status.sort_values().plot(kind = 'barh')
ax.set_title('Somatic Status')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
somatic_status_percentage = somatic_status*100/sum(somatic_status)
somatic_status_percentage.sort_values().plot(kind = 'barh')
ax.set_title('Somatic Status')
ax.set_xlabel('Percentage')
for container in ax.containers:
    ax.bar_label(container)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
specimen_preservation_type = msk_impact_clinical_data['Specimen Preservation Type'].value_counts()
specimen_preservation_type.plot(kind = 'barh')
ax.set_title('Type of specimen preservation')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)
    
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)  
specimen_preservation_type_percentage = specimen_preservation_type*100/sum(specimen_preservation_type)
specimen_preservation_type_percentage.sort_values().plot(kind = 'barh')
ax.set_title('Type of specimen preservation')
ax.set_xlabel('Percentage')
for container in ax.containers:
    ax.bar_label(container)

More than 82% of specimens are preserved as FFPE blocks

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
specimen_type = msk_impact_clinical_data['Specimen Type'].value_counts()
specimen_type.sort_values().plot(kind = 'barh')
ax.set_title('Specimen Type')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)

fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
specimen_type_percentage = specimen_type*100/sum(specimen_type)
specimen_type_percentage.sort_values().plot(kind = 'barh')
ax.set_title('Specimen Type')
ax.set_xlabel('Percentage')
for container in ax.containers:
    ax.bar_label(container)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
tmb = msk_impact_clinical_data['TMB (nonsynonymous)']
tmb.plot.hist(ax=ax,bins=12, alpha=0.5)
tmb.plot.kde(ax=ax1,c='k')
ax.set_xlabel('TMB (nonsynonymous)')

In [ ]:
print('Mean value of TMB(nonsynonymous): ', np.mean(tmb))
print('Std. dev of TMB(nonsynonymous): ', np.std(tmb))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
tumor_purity = msk_impact_clinical_data['Tumor Purity']
tumor_purity.plot.hist(ax=ax,bins=12, alpha=0.5)
tumor_purity.plot.kde(ax=ax1,c='k')
ax.set_xlabel('Tumor Purity')

In [ ]:
print('Mean value of Tumor Purity: ', np.mean(tumor_purity))
print('Std. dev of Tumor Purity: ', np.std(tumor_purity))

In [ ]:
msk_impact_clinical_data['Patient\'s Vital Status'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
patient_vital_status = msk_impact_clinical_data['Patient\'s Vital Status'].value_counts()
patient_vital_status.sort_values().plot(kind = 'barh')
ax.set_title('Patient\'s Vital Status')
ax.set_xlabel('Count/percentage')
for container in ax.containers:
    ax.bar_label(container)
    
patient_vital_status_percentage = patient_vital_status*100/sum(patient_vital_status)
patient_vital_status_percentage.sort_values().plot(kind = 'barh')
ax.set_xlabel('Count/percentage')
for container in ax.containers:
    ax.bar_label(container)

### EDA of data clinical patient

In [ ]:
data_clinical_patient.columns

In [ ]:
patient_identifier = data_clinical_patient['PATIENT_ID'].tolist()
print('No. of Patient Identifiers: ', len(patient_identifier))
print('No. of Unique Patient Identifiers: ', len(set(patient_identifier)))

### EDA of data clinical sample

In [ ]:
data_clinical_sample.head(2)

In [ ]:
data_clinical_sample.columns

### EDA of data cna hg19

In [ ]:
data_cna_hg19.head(2)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
data_cna_num_mark = data_cna_hg19['num.mark']
data_cna_num_mark.plot.hist(ax=ax,bins=12, alpha=0.5)
data_cna_num_mark.plot.kde(ax=ax1,c='k')
ax.set_xlabel('num.mark')

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
data_cna_seg_mean = data_cna_hg19['seg.mean']
data_cna_seg_mean.plot.hist(ax=ax,bins=12, alpha=0.5)
data_cna_seg_mean.plot.kde(ax=ax1,c='k')
ax.set_xlabel('seg.mean')

###  EDA of data gene panel matrix

In [ ]:
data_gene_panel_matrix.columns

In [ ]:
data_gene_panel_matrix.head(2)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
Mutations = data_gene_panel_matrix['mutations'].value_counts()
Mutations.sort_values().plot(kind = 'barh')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)
    
Mutations_percentage = Mutations*100/sum(Mutations)
Mutations_percentage.sort_values().plot(kind = 'barh')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)

It can be noted that all three columns 'mutations', 'cna', and 'structural_variants' have the same value distributions, hence only one plot is shown.

### EDA of data mutations

In [ ]:
data_mutations.columns

In [ ]:
data_mutations.head(2)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
variant_type = data_mutations['Variant_Type'].value_counts()
variant_type.sort_values().plot(kind = 'barh')
ax.set_title('Variant type')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)
    
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
variant_type_percentage = variant_type*100/sum(variant_type)
variant_type_percentage.sort_values().plot(kind = 'barh')
ax.set_xlabel('Percentage')
for container in ax.containers:
    ax.bar_label(container)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
protein_position = data_mutations['Protein_position']
protein_position.plot.hist(ax=ax,bins=12, alpha=0.5)
protein_position.plot.kde(ax=ax1,c='k')
ax.set_xlabel('Protein_position')

In [ ]:
print('Mean value of Protein_position: ', np.mean(protein_position))
print('Std. dev of Protein_position: ', np.std(protein_position))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
variant_classification = data_mutations['Variant_Classification'].value_counts()
variant_classification .sort_values().plot(kind = 'barh')
ax.set_title('Variant classification')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)
    
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)    
variant_classification_percentage = variant_classification *100/sum(variant_classification )
variant_classification_percentage.sort_values().plot(kind = 'barh')
ax.set_xlabel('Percentage')
for container in ax.containers:
    ax.bar_label(container)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
t_ref_count = data_mutations['t_ref_count']
t_ref_count.plot.hist(ax=ax,bins=12, alpha=0.5)
t_ref_count.plot.kde(ax=ax1,c='k')
ax.set_xlabel('t_ref_count')

In [ ]:
print('Mean value of t_ref_count: ', np.mean(t_ref_count))
print('Std. dev of t_ref_count: ', np.std(t_ref_count))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
t_alt_count = data_mutations['t_alt_count']
t_alt_count.plot.hist(ax=ax,bins=12, alpha=0.5)
t_alt_count.plot.kde(ax=ax1,c='k')
ax.set_xlabel('t_alt_count')

In [ ]:
print('Mean value of t_alt_count: ', np.mean(t_alt_count))
print('Std. dev of t_alt_count: ', np.std(t_alt_count))

In [ ]:
data_mutations['Codons'].value_counts()

### EDA of data_SV

In [ ]:
data_sv.columns

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
tumor_read_count = data_sv['Tumor_Read_Count']
tumor_read_count.plot.hist(ax=ax,bins=12, alpha=0.5)
tumor_read_count.plot.kde(ax=ax1,c='k')
ax.set_xlabel('Tumor_Read_Count')

In [ ]:
print('Mean value of tumor_read_count: ', np.mean(tumor_read_count))
print('Std. dev of tumor_read_count: ', np.std(tumor_read_count))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
tumor_variant_count = data_sv['Tumor_Variant_Count']
tumor_variant_count.plot.hist(ax=ax,bins=12, alpha=0.5)
tumor_variant_count.plot.kde(ax=ax1,c='k')
ax.set_xlabel('Tumor_Variant_Count')

In [ ]:
print('Mean value of tumor_variant_count: ', np.mean(tumor_variant_count))
print('Std. dev of tumor_variant_count: ', np.std(tumor_variant_count))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
breakpoint_type = data_sv['Breakpoint_Type'].value_counts()
breakpoint_type.sort_values().plot(kind = 'barh')
ax.set_title('Breakpoint type')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)
    
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
breakpoint_type_percentage = breakpoint_type  *100/sum(breakpoint_type )
breakpoint_type_percentage.sort_values().plot(kind = 'barh')
ax.set_xlabel('Percentage')
for container in ax.containers:
    ax.bar_label(container)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
normal_variant_count = data_sv['Normal_Variant_Count'].value_counts()
normal_variant_count.sort_values().plot(kind = 'barh')
ax.set_title('Normal variant count')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)
    
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)   
normal_variant_count_percentage = normal_variant_count  *100/sum(normal_variant_count )
normal_variant_count_percentage.sort_values().plot(kind = 'barh')
ax.set_xlabel('Percentage')
for container in ax.containers:
    ax.bar_label(container)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
sv_length = data_sv['SV_Length']
sv_length.plot.hist(ax=ax,bins=12, alpha=0.5)
sv_length.plot.kde(ax=ax1,c='k')
ax.set_xlabel('SV_Length')

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
normal_split_read_count = data_sv['Normal_Split_Read_Count']
normal_split_read_count.plot.hist(ax=ax,bins=12, alpha=0.5)
normal_split_read_count.plot.kde(ax=ax1,c='k')
ax.set_xlabel('Normal_Split_Read_Count')

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
normal_paired_end_read_count = data_sv['Normal_Paired_End_Read_Count']
normal_paired_end_read_count.plot.hist(ax=ax,bins=12, alpha=0.5)
normal_paired_end_read_count.plot.kde(ax=ax1,c='k')
ax.set_xlabel('Normal_Paired_End_Read_Count')

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
ax1 = ax.twinx()
normal_read_count = data_sv['Normal_Read_Count']
normal_read_count.plot.hist(ax=ax,bins=12, alpha=0.5)
normal_read_count.plot.kde(ax=ax1,c='k')
ax.set_xlabel('Normal_Read_Count')

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
connection_type = data_sv['Connection_Type'].value_counts()
connection_type.sort_values().plot(kind = 'barh')
ax.set_title('Connection type')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)
    
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)   
connection_type_percentage = connection_type*100/sum(connection_type)
connection_type_percentage.sort_values().plot(kind = 'barh')
ax.set_xlabel('Percentage')
for container in ax.containers:
    ax.bar_label(container)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)
class_info = data_sv['Class'].value_counts()
class_info.sort_values().plot(kind = 'barh')
ax.set_title('Class')
ax.set_xlabel('Count')
for container in ax.containers:
    ax.bar_label(container)
    
fig, ax = plt.subplots(figsize=(4, 3), dpi=120)   
class_info_percentage = class_info*100/sum(class_info)
class_info_percentage.sort_values().plot(kind = 'barh')
ax.set_xlabel('Percentage')
for container in ax.containers:
    ax.bar_label(container)

## Summary

The following features have been explored and plotted from various datasets in the MSK-IMPACT (Memorial Sloan Kettering-Integrated Mutation Profiling of Actionable Cancer Targets) study

1. "Patient's Vital Status"
2. '#Patient Identifier'
3. 'Breakpoint_Type'
4. 'Cancer Type'
5. 'Class'
6. 'Codons'
7. 'DNA Input'
8. 'Fraction Genome Altered'
9. 'Matched Status'
10. 'Metastatic Site'
11. 'Mutation Count'
12. 'mutations'
13. 'Normal_Paired_End_Read_Count'
14. 'Normal_Read_Count'
15. 'Normal_Split_Read_Count'
16. 'Normal_Variant_Count'
17. 'num.mark'
18. 'Number of Samples Per Patient'
19. 'Overall Survival (Months)'
20. 'Overall Survival Status'
21. 'Patient ID'
22. 'Primary Tumor Site'
23. 'Protein_position'
24. 'Sample Collection Source'
25. 'Sample coverage'
26. 'Sample ID'
27. 'Sample Type'
28. 'seg.mean'
29. 'Sex'
30. 'Smoking History'
31. 'Somatic Status'
32. 'Specimen Preservation Type'
33. 'Specimen Type'
34. 'SV_Length'
35. 't_alt_count'
36. 't_ref_count'
37. 'TMB (nonsynonymous)'
38. 'Tumor Purity'
39. 'Tumor_Read_Count'
40. 'Tumor_Variant_Count'
41. 'Variant_Classification'
42. 'Variant_Type'
43. 'Connection_Type'

### Observations

The following observations were made based on the exploratory data analysis

1. It can be seen that 71.55% of patients are alive and 28.45% patients are deceased
2. 65.04% of samples are collected In-House and 34.96% of samples are collected outside home
3. Only one sample is collected from majority (89%) of patients
4. 43.23% of samples are that of metastasis stage
5. Gender distribution of patients is more or less equal (i.e. 50%-50%)
6. More than 82% of specimens are preserved as FFPE blocks
7. Most frequenst codon change is Cga/Tga 
8. Most frequent class in deletion
9. Most frequent connection type is 3to5
10. Overall Survival Months feature and Cancer type feature has missing values of 25% and 15%. Hence these features can be removed
11. Certain features are highly correlated (both positive/negative correlation with each other)
13. Density plots of numerical features between target variable shows some disinction between probability density graphs
12. Three statistically significant features 'SAMPLE_COVERAGE', 'DNAInput_ng', 'SCNACount' with an alpha level of 0.001 were foundout
13. The violion plots of significant features with respect to target variable shows some difference
13. There is a good number of outliers (|z-score|>3) among numerical features
14. TSNE plot shows some difference between both classes
15. A baseline accuracy of 57.94% is expected